### Scrapping Completo

In [1]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import pandas as pd
import re

In [3]:

# 1. Función para extraer datos genéricos desde etiquetas HTML
def extraer_datos(soup, tag, attrs):
    """
    Extrae el texto de todas las etiquetas HTML coincidentes con el tag y los atributos dados.

    Parameters:
    soup (bs4.BeautifulSoup): Objeto BeautifulSoup del contenido HTML.
    tag (str): El nombre de la etiqueta HTML que deseas buscar.
    attrs (dict): Un diccionario de atributos para identificar las etiquetas.

    Returns:
    list: Lista con los textos extraídos.
    """
    elementos = soup.find_all(tag, attrs=attrs)
    return [elemento.text.strip() if elemento else None for elemento in elementos]

# 2. Función para normalizar nombres de columnas
def normalizar_nombres_columnas(df):
    """
    Normaliza los nombres de las columnas del DataFrame, convirtiéndolos a mayúsculas.
    """
    df.columns = df.columns.str.upper()
    return df

# 3. Función para Encontrar caracteres: Funcion genérica para adaptarla
def buscar_numero_en(df, col_donde_sebusca, antes_de=None, despues_de=None):
    """
    Busca un número en la columna `col_metraje` de un DataFrame, especificando un patrón antes y/o después del número.
    
    Parameters:
    df (pd.DataFrame): El DataFrame donde se realiza la búsqueda.
    col_metraje (str): El nombre de la columna donde buscar el patrón.
    antes_de (str): El patrón que aparece antes del número (opcional).
    despues_de (str): El patrón que aparece después del número (opcional).
    
    Returns:
    pd.Series: Serie con los números encontrados.
    """
    
    # Crear la expresión regular
    # Si no se especifica "antes_de", solo busca basado en "despues_de"
    # Si no se especifica "despues_de", solo busca basado en "antes_de"
    if antes_de and despues_de:
        regex = rf'{re.escape(despues_de)}\s*(\d+)\s*{re.escape(antes_de)}'
    elif despues_de:
        regex = rf'{re.escape(despues_de)}\s*(\d+)'
    elif antes_de:
        regex = rf'(\d+)\s*{re.escape(antes_de)}'
    else:
        raise ValueError("Debes especificar al menos 'antes_de' o 'despues_de'.")
    
    # Aplicar la búsqueda de la expresión regular en la columna especificada
    return df[col_donde_sebusca].apply(lambda x: re.search(regex, x).group(1) if re.search(regex, x) else None)

# 4. Función para buscar un número entre patrones opcionales
def buscar_numero_en_precio(df, col_donde_sebusca, antes_de=None, despues_de=None):
    """
    Busca un número en la columna `col_metraje` de un DataFrame, especificando un patrón antes y/o después del número.
    Convierte números con separadores de miles (e.g., "125,500") en enteros (125500).
    
    Parameters:
    df (pd.DataFrame): El DataFrame donde se realiza la búsqueda.
    col_metraje (str): El nombre de la columna donde buscar el patrón.
    antes_de (str): El patrón que aparece antes del número (opcional).
    despues_de (str): El patrón que aparece después del número (opcional).
    
    Returns:
    pd.Series: Serie con los números encontrados.
    """
    
    # Crear la expresión regular para encontrar el número, considerando los patrones antes y/o después
    if antes_de and despues_de:
        regex = rf'{re.escape(despues_de)}\s*(\d{{1,3}}(?:,\d{{3}})*)\s*{re.escape(antes_de)}'
    elif despues_de:
        regex = rf'{re.escape(despues_de)}\s*(\d{{1,3}}(?:,\d{{3}})*)'
    elif antes_de:
        regex = rf'(\d{{1,3}}(?:,\d{{3}})*)\s*{re.escape(antes_de)}'
    else:
        raise ValueError("Debes especificar al menos 'antes_de' o 'despues_de'.")

    # Aplicar la expresión regular y limpiar el número
    return df[col_donde_sebusca].apply(lambda x: int(re.sub(r',', '', re.search(regex, x).group(1))) if re.search(regex, x) else None)


# 5. Función para limpiar precios y extraer monedas
def limpiar_precio(df, col_precio='PRECIO_DESDE'):
    """
    Limpia la columna de precios, detectando la moneda (Soles o Dólares) y extrayendo los valores numéricos.
    """
    df['Precio'] = df[col_precio].astype(str)
    df['Moneda_soles'] = df['Precio'].apply(lambda x: "Soles" if 'S/' in x else None)
    df['Moneda_dolares'] = df['Precio'].apply(lambda x: "Dólares" if 'USD' in x else None)
    
    #df['Precio_soles'] = df['Precio'].apply(lambda x: re.findall(r'\d+', x)[0] if 'S/' in x else None)
    df['Precio_soles'] = df.apply(lambda x: re.findall(r'\d+', x['Precio'])[0] if x['Moneda_soles'] else None, axis=1)

    #df['Precio_dolares'] = df['Precio'].apply(lambda x: re.findall(r'\d+', x)[0] if 'USD' in x else None)
    df['Precio_dolares'] = buscar_numero_en_precio(df, col_donde_sebusca = col_precio, despues_de='USD')
    
    # Convertir a tipo numérico
    df['Precio_soles'] = pd.to_numeric(df['Precio_soles'], errors='coerce')*1000.0
    df['Precio_dolares'] = pd.to_numeric(df['Precio_dolares'], errors='coerce')
    
    # Eliminar cualquier columna adicional si es necesario
    if 'Precio' in df.columns:
        df.drop(columns=['Precio'], inplace=True)
    
    return df

# 6. Función para limpiar ubicación
def limpiar_ubicacion(df, col_ubicacion='UBICACION', distritos=None):
    """
    Limpia la columna de ubicación eliminando distritos no deseados y caracteres especiales.
    """
    if distritos is None:
        distritos = ['Comas','COMAS','comas', 'Breña','BREÑA', 'Lima', 'Peru', 'Perú', 'SAN MIGUEL', 'LIMA', 'PERU']
    
    def limpiar_calle(ubicacion, distritos):
        for distrito in distritos:
            ubicacion = re.sub(rf'\b{distrito}\b', '', ubicacion) # Usamos \b para asegurarnos de que elimine el distrito cuando sea una palabra completa
            ubicacion = re.sub(distrito, '', ubicacion) # Eliminamos los casos donde el distrito pueda estar pegado a otras palabras
        
        ubicacion = re.sub(r'[.,;:!?\s]+$', '', ubicacion).strip()  # Eliminar cualquier signo de puntuación al final de la cadena
        ubicacion = re.sub(r'\s+', ' ', ubicacion).strip() # Eliminar espacios extras
        return ubicacion if ubicacion else None

    df['Calle'] = df[col_ubicacion].apply(lambda x: limpiar_calle(x, distritos) if isinstance(x, str) else None)
    return df

# 7. Función para extraer metraje y características adicionales
def extraer_metraje(df, col_metraje='METRAJE'):
    """
    Extrae información de metraje, unidades y dormitorios de la columna metraje.
    """
    """ df['Metraje'] = df[col_metraje].apply(lambda x: re.search(r'\d+ m²', x).group() if isinstance(x, str) else None)
    df['Dormitorios'] = df[col_metraje].apply(lambda x: re.search(r'\d+ dorm.', x).group() if isinstance(x, str) else None)
    return df """
    # Usando la función genérica para extraer diferentes valores
    df['Unidades'] = buscar_numero_en(df, col_metraje, antes_de='un.')
    df['Dormitorios'] = buscar_numero_en(df, col_metraje, antes_de='dorm.')
    df['Metraje_desde'] = buscar_numero_en(df, col_metraje, antes_de='a', despues_de='dorm.')
    df['Metraje_hasta'] = buscar_numero_en(df, col_metraje, antes_de='m²')
    df['Baños'] = buscar_numero_en(df, col_metraje, antes_de='baño')
    df['Estacionamientos'] = buscar_numero_en(df, col_metraje, antes_de='esta')
    
    # Convertir los resultados a numéricos donde sea necesario
    df['Unidades'] = pd.to_numeric(df['Unidades'], errors='coerce')
    df['Dormitorios'] = pd.to_numeric(df['Dormitorios'], errors='coerce')
    df['Metraje_desde'] = pd.to_numeric(df['Metraje_desde'], errors='coerce')
    df['Metraje_hasta'] = pd.to_numeric(df['Metraje_hasta'], errors='coerce')
    df['Baños'] = pd.to_numeric(df['Baños'], errors='coerce')
    df['Estacionamientos'] = pd.to_numeric(df['Estacionamientos'], errors='coerce')

    print(df.info())
    return df

# 8. Función para Cálculos de S/ o $ por m2
def añadir_calculos(df):
    df['Precio por m2 soles ref desde'] = round(df['Precio_soles']/df['Metraje_hasta'],0)
    df['Precio por m2 soles ref hasta'] = round(df['Precio_soles']/df['Metraje_desde'],0)
    
    if df['Precio_dolares'] is not None:
        df['Precio por m2 dol ref desde'] = round(df['Precio_dolares']/df['Metraje_hasta'],0)
        df['Precio por m2 dol ref hasta'] = round(df['Precio_dolares']/df['Metraje_desde'],0)
    
    return df    

# 9. Función para exportar a Excel

def exportar_excel(df, nombre_file, proyecto):
    DATA_ANALYTICS = "C:/Users/diego.dinatale/OneDrive - Aenza/LOCAL_MACHINE/DATA_ANALYTICS/"
    #BDVIVA = "C:/Users/diego.dinatale/OneDrive - Aenza/LOCAL_MACHINE/DATA_ANALYTICS/BD VIVA/ESTUDIOS DE PROYECTOS/BUSINESS INTELLIGENCE CON SCRAPPING/SCRAPPING DE PDMAR/BD de Internet Jugadores/"
    nombre_file = nombre_file + " "+ proyecto
    ubicacion_final_baseglobal = DATA_ANALYTICS + "BD VIVA/DATA DEL MERCADO CON SCRAPPING/CERCADO DE LIMA/" + nombre_file + ".xlsx"
    #ubicacion_final_carpeta_proyecto = DATA_ANALYTICS + "ESTUDIOS DE PROYECTOS/BUSINESS INTELLIGENCE CON SCRAPPING/SCRAPPING DE LINCE/BD de Internet Jugadores/" + nombre_file + ".xlsx"
    #DATA DEL MERCADO CON SCRAPPING
    print(f"Exportando archivo a: {ubicacion_final_baseglobal}")
    df.to_excel(ubicacion_final_baseglobal, index=False)
    #print(f"Exportando archivo a: {ubicacion_final_carpeta_proyecto}")
    #df.to_excel(ubicacion_final_carpeta_proyecto, index=False)
    print("Archivo exportado exitosamente.")

# 8. Función principal de scraping y creación del DataFrame
def scraping_selenium(url):
    driver = webdriver.Chrome()
    driver.get(url)
    
    contenido = driver.page_source
    soup = bs(contenido, 'html.parser')
    driver.quit() # Cerrar navegador
    
    # Extraer datos usando las funciones genéricas
    titulo_sucio = extraer_datos(soup, "div", {"class": "caption"}) # 1
    
    # Crear el DataFrame
    data = {
        "TITULO": titulo_sucio
    }
    df = pd.DataFrame(data)

    # Limpiar y procesar los datos
    df = normalizar_nombres_columnas(df)
    return df



<>:135: SyntaxWarning: invalid escape sequence '\d'
<>:135: SyntaxWarning: invalid escape sequence '\d'
C:\Users\diego.dinatale\AppData\Local\Temp\ipykernel_27888\3284646049.py:135: SyntaxWarning: invalid escape sequence '\d'
  """ df['Metraje'] = df[col_metraje].apply(lambda x: re.search(r'\d+ m²', x).group() if isinstance(x, str) else None)


# DETALLAR

In [19]:
from bs4 import BeautifulSoup
from selenium import webdriver

url1 = "https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2825-gran-central-colonial-ii-cercado-de-lima-lima-lima-los-portales-departamentos"

# 1. Levantar el driver y obtener el HTML
driver = webdriver.Chrome()
driver.get(url1)
html = driver.page_source
driver.quit()

# 2. Parsear con BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# 3. Imprimir el HTML completo (o parte de él)
print(soup.prettify()[:1000])   # solo los primeros 1.000 caracteres

# 4. Ver todas las etiquetas <div> con clase "caption"
for div in soup.find_all("div", class_="caption"):
    print(div)
    print("—" * 40)

# 5. Buscar y mostrar todas las etiquetas que contengan la palabra ‘Dormitorio’
import re
for tag in soup.find_all(text=re.compile(r"Dormitorios?", re.IGNORECASE)):
    print(tag.parent)  # muestra el elemento padre HTML donde aparece el texto
    print("—" * 40)


<html class="js flexbox canvas canvastext webgl no-touch geolocation postmessage no-websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers no-applicationcache svg inlinesvg smil svgclippaths" lang="es">
 <head>
  <meta content="A7vZI3v+Gz7JfuRolKNM4Aff6zaGuT7X0mf3wtoZTnKv6497cVMnhy03KDqX7kBz/q/iidW7srW31oQbBt4VhgoAAACUeyJvcmlnaW4iOiJodHRwczovL3d3dy5nb29nbGUuY29tOjQ0MyIsImZlYXR1cmUiOiJEaXNhYmxlVGhpcmRQYXJ0eVN0b3JhZ2VQYXJ0aXRpb25pbmczIiwiZXhwaXJ5IjoxNzU3OTgwODAwLCJpc1N1YmRvbWFpbiI6dHJ1ZSwiaXNUaGlyZFBhcnR5Ijp0cnVlfQ==" http-equiv="origin-trial"/>
  <style>
   #nav-flat,#nav-duplex{    height: auto !important;}.ficha_proyecto1  .label-success {    background: #ECFDF3;    color: #12B76A;    padding: .4em .8em .4em;   

C:\Users\diego.dinatale\AppData\Local\Temp\ipykernel_27888\2877309385.py:25: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for tag in soup.find_all(text=re.compile(r"Dormitorios?", re.IGNORECASE)):


In [ ]:
import re
import pandas as pd
from bs4 import BeautifulSoup as BS
from selenium import webdriver

class MarketScraper:
    """
    Clase para extracción, limpieza y análisis de datos de proyectos inmobiliarios.
    """
    def __init__(self, url=None, df=None):
        self.url = url
        self.df = df
        self.soup = None

    def _init_driver(self):
        driver = webdriver.Chrome()
        driver.get(self.url)
        content = driver.page_source
        driver.quit()
        self.soup = BS(content, 'html.parser')

    def extract_data(self, tag: str, attrs: dict) -> list:
        items = self.soup.find_all(tag, attrs=attrs)
        return [el.text.strip() for el in items]

    def normalize_columns(self):
        self.df.columns = self.df.columns.str.upper()
        return self

    def debug_dormitorios(self) -> list:
        """
        Método de depuración: devuelve el HTML de los elementos que contienen 'Dormitorios'.
        Útil para inspeccionar etiquetas antes de ajustar la extracción.
        """
        # Busca cualquier texto que contenga 'Dormitorio' (con o sin 's')
        matches = self.soup.find_all(text=re.compile(r'Dormitoris?', re.IGNORECASE))
        # Devuelve cadena HTML de cada elemento padre
        return [str(m.parent) for m in matches]

    def extract_title_features(self, title_col: str = 'TITULO'):
        def _parse(titulo):
            u = re.search(r'(\d+) unidad?', titulo)
            unidades = int(u.group(1)) if u else None
            m = re.search(r'(\w+)\sPiso', titulo)
            modelo = m.group(1) if m else None
            ep = re.search(r'Entre\s*(\d+)\s*al\s*(\d+)', titulo)
            if ep:
                piso_desde, piso_hasta = int(ep.group(1)), int(ep.group(2))
                total_pisos = piso_hasta - piso_desde + 1
            else:
                pc = re.findall(r'Piso (\d+(?:, \d+)*)', titulo)
                if pc:
                    pisos_list = [int(p) for p in pc[0].split(', ')]
                    piso_desde, piso_hasta = min(pisos_list), max(pisos_list)
                    total_pisos = len(pisos_list)
                else:
                    piso_desde = piso_hasta = total_pisos = None
            d = re.search(r'(\d+) Dormitorios?', titulo)
            dormitorios = int(d.group(1)) if d else None
            a = re.search(r'Área ([\d.]+)', titulo)
            area = float(a.group(1)) if a else None
            p = re.search(r'Precio desde S/ ([\d,]+)', titulo)
            precio = float(p.group(1).replace(',', '')) if p else None
            return [unidades, modelo, piso_desde, piso_hasta, total_pisos, dormitorios, area, precio]
        cols = ['Unidades_Disponibles', 'Modelo', 'Piso_Desde', 'Piso_Hasta', 'Total_Pisos',
                'Dormitorios', 'Area_m2', 'Precio_Desde_Soles']
        title_df = pd.DataFrame(self.df[title_col].astype(str).apply(_parse).tolist(), columns=cols)
        self.df = pd.concat([self.df, title_df], axis=1)
        self.df.dropna(how='all', subset=cols, inplace=True)
        return self

    def _build_regex(self, before: str = None, after: str = None, price: bool = False) -> str:
        num = r"\d{1,3}(?:,\d{3})*" if price else r"\d+"
        if before and after:
            return rf"{re.escape(after)}\s*({num})\s*{re.escape(before)}"
        if after:
            return rf"{re.escape(after)}\s*({num})"
        if before:
            return rf"({num})\s*{re.escape(before)}"
        raise ValueError("Se requiere al menos 'before' o 'after'.")

    def find_number(self, col: str, before: str = None, after: str = None, price: bool = False) -> pd.Series:
        regex = self._build_regex(before, after, price)
        return self.df[col].apply(lambda x: int(re.search(regex, str(x)).group(1).replace(',', ''))
                                  if re.search(regex, str(x)) else None)

    def clean_price(self, col: str = 'PRECIO_DESDE'):
        if col in self.df.columns:
            s = self.df[col].astype(str)
            self.df['Moneda_soles'] = s.str.contains('S/').map({True: 'Soles', False: None})
            self.df['Moneda_dolares'] = s.str.contains('USD').map({True: 'Dólares', False: None})
            self.df['Precio_soles'] = s.where(self.df['Moneda_soles'].notnull()) \
                .str.extract(r"(\d+)\b")[0].astype(float) * 1000
            self.df['Precio_dolares'] = self.find_number(col, after='USD', price=True)
        return self

    def clean_location(self, col: str = 'UBICACION', districts: list = None):
        if col in self.df.columns:
            if districts is None:
                districts = ['Comas', 'Breña', 'Lima', 'Peru', 'Perú', 'San Miguel']
            def _clean(text):
                if not isinstance(text, str):
                    return None
                for d in districts:
                    text = re.sub(rf"\b{d}\b", '', text, flags=re.IGNORECASE)
                return re.sub(r"\s+", ' ', re.sub(r"[\.,;:!?]+$", '', text)).strip() or None
            self.df['Calle'] = self.df[col].apply(_clean)
        return self

    def extract_features(self, col: str = 'METRAJE'):
        if col in self.df.columns:
            keys = ['Unidades', 'Dormitorios', 'Metraje_desde', 'Metraje_hasta', 'Baños', 'Estacionamientos']
            patterns = [('un.', None), ('dorm.', None), ('a', 'dorm.'), ('m²', None), ('baño', None), ('esta', None)]
            for key, (before, after) in zip(keys, patterns):
                self.df[key] = pd.to_numeric(self.find_number(col, before=before, after=after), errors='coerce')
        return self

    def add_calculations(self):
        if 'Precio_soles' in self.df and 'Metraje_hasta' in self.df:
            self.df['Precio m2_soles_desde'] = (self.df['Precio_soles'] / self.df['Metraje_hasta']).round()
        if 'Precio_soles' in self.df and 'Metraje_desde' in self.df:
            self.df['Precio m2_soles_hasta'] = (self.df['Precio_soles'] /


In [18]:
# Ejecutar la función principal con la URL deseada

# NEXO INMOBILIARIA
# 41 dptos
#url = "https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2907-alto-lima-park-cercado-de-lima-lima-lima-imagina"
url = "https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2825-gran-central-colonial-ii-cercado-de-lima-lima-lima-los-portales-departamentos"

""" url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2825-gran-central-colonial-ii-cercado-de-lima-lima-lima-los-portales-departamentos"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3737-nova-cercado-de-lima-lima-lima-kallpa"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2213-checor-las-magnolias-cercado-de-lima-lima-lima-checor-inmobiliaria"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2715-central-park-cercado-de-lima-lima-lima-tribeca-inmobiliaria"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3008-amatista-cercado-de-lima-lima-lima-marte"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2660-alto-alborada-cercado-de-lima-lima-lima-imagina"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3035-jardines-de-santa-beatriz-etapa-ii-cercado-de-lima-lima-lima-tm-gestion-inmobiliaria"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3385-albamar-re-cercado-de-lima-lima-lima-albamar-grupo-inmobiliario"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2888-duetto-846-fase-ii-cercado-de-lima-lima-lima-cantabria"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3053-minimal-cercado-de-lima-lima-lima-grupo-tyc"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3381-novo-3-cercado-de-lima-lima-lima-my-home"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3011-torre-castelo-cercado-de-lima-lima-lima-capac-asociados"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-1993-s14-apartments-cercado-de-lima-lima-lima-urbana-per%C3%BA"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2641-checor-las-dalias-cercado-de-lima-lima-lima-checor-inmobiliaria"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3186-helio-santa-beatriz-cercado-de-lima-lima-lima-padova"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3027-qantua-fase-1-cercado-de-lima-lima-lima-grupo-lar"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3321-centriqo-club-condomino-ecoamigable-cercado-de-lima-lima-lima-besco"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2912-nueva-metropolis-cercado-de-lima-lima-lima-invent"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2996-albamar-arenales-cercado-de-lima-lima-lima-albamar-grupo-inmobiliario"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2767-ciudaris-i-iconik-park-cercado-de-lima-lima-lima-ciudaris"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3693-roble-cercado-de-lima-lima-lima-abril-grupo-inmobiliario"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3453-cedro-33-cercado-de-lima-lima-lima-abril-grupo-inmobiliario"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2647-alameda-central-condominio-cercado-de-lima-lima-lima-besco"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2727-concepto-urban-park-cercado-de-lima-lima-lima-imagina"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3684-albamar-bea-cercado-de-lima-lima-lima-albamar-grupo-inmobiliario"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3001-urban-tower-3-cercado-de-lima-lima-lima-quatro-inmobiliaria"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3452-qantua-fase-2-cercado-de-lima-lima-lima-grupo-lar"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2324-park-living-cercado-de-lima-lima-lima-eureka"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2462-duetto-846-cercado-de-lima-lima-lima-cantabria"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2850-pietra-centrale-cercado-de-lima-lima-lima-rumi-grupo-inmobiliario"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3491-centrico-cercado-de-lima-lima-lima-grupo-mg"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-1867-edificio-vista-verde-cercado-de-lima-lima-lima-grupo-mg"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2253-invent-tirado-cercado-de-lima-lima-lima-invent"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2907-alto-lima-park-cercado-de-lima-lima-lima-imagina"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2308-smart-a-cercado-de-lima-lima-lima-grupo-mg"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3209-tulipan-cercado-de-lima-lima-lima-abril-grupo-inmobiliario"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3298-cool-living-cercado-de-lima-lima-lima-promsal"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-1816-central-home-cercado-de-lima-lima-lima-casa-bonita"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2965-santorini-cercado-de-lima-lima-lima-brazil-grupo-inmobiliario"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3466-evolution-cercado-de-lima-lima-lima-pisonay-inmobiliaria"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-1866-edificio-los-pinos-iii-cercado-de-lima-lima-lima-grupo-mg"
"""

df = scraping_selenium(url)

# Actualización de la función para manejar los dos patrones adicionales
def limpiar_titulo_actualizado(titulo):
        # Buscar número de unidades disponibles
        #unidades = re.search(r'(\d+) unidades?', titulo)
        unidades = re.search(r'(\d+) unidad?', titulo)
        unidades_disponibles = int(unidades.group(1)) if unidades else None
        
        # Buscar el modelo (secuencia de palabras hasta 'Piso')
        modelo = re.search(r'(\w+)\sPiso', titulo)
        modelo_nombre = modelo.group(1) if modelo else None

        # Caso 1: Patrón "Entre X al Y"
        # entre_pisos = re.search(r'Entre (\d+) al (\d+)', titulo)
        entre_pisos = re.search(r'Entre\s*(\d+)\s*al\s*(\d+)', titulo)
        if entre_pisos:
            piso_desde = int(entre_pisos.group(1))
            piso_hasta = int(entre_pisos.group(2))
            total_pisos = piso_hasta - piso_desde + 1
        else:
            # Caso 2: Patrón "Piso X, Y, Z"
            pisos_comas = re.findall(r'Piso (\d+(?:, \d+)*)', titulo)
            if pisos_comas:
                pisos = [int(piso) for piso in pisos_comas[0].split(', ')]
                piso_desde = min(pisos)
                piso_hasta = max(pisos)
                total_pisos = len(pisos)
            else:
                piso_desde, piso_hasta, total_pisos = None, None, None

        # Buscar dormitorios
        dormitorios = re.search(r'(\d+) Dormitorios?', titulo)
        dormitorios_num = int(dormitorios.group(1)) if dormitorios else None
        
        # Buscar área
        area = re.search(r'Área ([\d.]+)', titulo)
        area_m2 = float(area.group(1)) if area else None
        
        # Buscar precio
        precio = re.search(r'Precio desde S/ ([\d,]+)', titulo)
        precio_soles = float(precio.group(1).replace(',', '')) if precio else None
        
        return [unidades_disponibles, modelo_nombre, piso_desde, piso_hasta, total_pisos, dormitorios_num, area_m2, precio_soles]

    # Aplicar la función actualizada

df_comas_cleaned_actualizado = df['TITULO'].apply(limpiar_titulo_actualizado)
df_comas_cleaned_actualizado = pd.DataFrame(df_comas_cleaned_actualizado.tolist(), columns=[
    'Unidades_Disponibles', 'Modelo', 'Piso_Desde', 'Piso_Hasta', 'Total_Pisos', 'Dormitorios', 'Area_m2', 'Precio_Desde_Soles'
])

df_comas_cleaned_actualizado = df_comas_cleaned_actualizado.dropna(axis=0, how='all')

exportar_excel(df_comas_cleaned_actualizado, nombre_file="Lima Proyecto", proyecto = "Gran Central Colonial II 26_05_25")

# Mostrar el DataFrame resultante
print(df.head())

Exportando archivo a: C:/Users/diego.dinatale/OneDrive - Aenza/LOCAL_MACHINE/DATA_ANALYTICS/BD VIVA/DATA DEL MERCADO CON SCRAPPING/CERCADO DE LIMA/Lima Proyecto Gran Central Colonial II 26_05_25.xlsx
Archivo exportado exitosamente.
                                              TITULO
0  7 unidades disponibles                        ...
1  5 unidades disponibles                        ...
2  13 unidades disponibles                       ...
3  28 unidades disponibles                       ...
4  HELIO SANTA BEATRIZ  Desde S/. 244,600  Enriqu...


In [ ]:
# Ejecutar la función principal con la URL deseada

# NEXO INMOBILIARIA
# 41 dptos
#url = "https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2907-alto-lima-park-cercado-de-lima-lima-lima-imagina"
url = "https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2825-gran-central-colonial-ii-cercado-de-lima-lima-lima-los-portales-departamentos"

url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2825-gran-central-colonial-ii-cercado-de-lima-lima-lima-los-portales-departamentos"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3737-nova-cercado-de-lima-lima-lima-kallpa"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2213-checor-las-magnolias-cercado-de-lima-lima-lima-checor-inmobiliaria"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2715-central-park-cercado-de-lima-lima-lima-tribeca-inmobiliaria"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3008-amatista-cercado-de-lima-lima-lima-marte"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2660-alto-alborada-cercado-de-lima-lima-lima-imagina"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3035-jardines-de-santa-beatriz-etapa-ii-cercado-de-lima-lima-lima-tm-gestion-inmobiliaria"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3385-albamar-re-cercado-de-lima-lima-lima-albamar-grupo-inmobiliario"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2888-duetto-846-fase-ii-cercado-de-lima-lima-lima-cantabria"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3053-minimal-cercado-de-lima-lima-lima-grupo-tyc"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3381-novo-3-cercado-de-lima-lima-lima-my-home"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3011-torre-castelo-cercado-de-lima-lima-lima-capac-asociados"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-1993-s14-apartments-cercado-de-lima-lima-lima-urbana-per%C3%BA"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2641-checor-las-dalias-cercado-de-lima-lima-lima-checor-inmobiliaria"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3186-helio-santa-beatriz-cercado-de-lima-lima-lima-padova"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3027-qantua-fase-1-cercado-de-lima-lima-lima-grupo-lar"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3321-centriqo-club-condomino-ecoamigable-cercado-de-lima-lima-lima-besco"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2912-nueva-metropolis-cercado-de-lima-lima-lima-invent"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2996-albamar-arenales-cercado-de-lima-lima-lima-albamar-grupo-inmobiliario"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2767-ciudaris-i-iconik-park-cercado-de-lima-lima-lima-ciudaris"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3693-roble-cercado-de-lima-lima-lima-abril-grupo-inmobiliario"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3453-cedro-33-cercado-de-lima-lima-lima-abril-grupo-inmobiliario"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2647-alameda-central-condominio-cercado-de-lima-lima-lima-besco"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2727-concepto-urban-park-cercado-de-lima-lima-lima-imagina"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3684-albamar-bea-cercado-de-lima-lima-lima-albamar-grupo-inmobiliario"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3001-urban-tower-3-cercado-de-lima-lima-lima-quatro-inmobiliaria"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3452-qantua-fase-2-cercado-de-lima-lima-lima-grupo-lar"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2324-park-living-cercado-de-lima-lima-lima-eureka"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2462-duetto-846-cercado-de-lima-lima-lima-cantabria"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2850-pietra-centrale-cercado-de-lima-lima-lima-rumi-grupo-inmobiliario"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3491-centrico-cercado-de-lima-lima-lima-grupo-mg"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-1867-edificio-vista-verde-cercado-de-lima-lima-lima-grupo-mg"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2253-invent-tirado-cercado-de-lima-lima-lima-invent"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2907-alto-lima-park-cercado-de-lima-lima-lima-imagina"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2308-smart-a-cercado-de-lima-lima-lima-grupo-mg"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3209-tulipan-cercado-de-lima-lima-lima-abril-grupo-inmobiliario"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3298-cool-living-cercado-de-lima-lima-lima-promsal"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-1816-central-home-cercado-de-lima-lima-lima-casa-bonita"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2965-santorini-cercado-de-lima-lima-lima-brazil-grupo-inmobiliario"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3466-evolution-cercado-de-lima-lima-lima-pisonay-inmobiliaria"
url ="https://nexoinmobiliario.pe/proyecto/venta-de-departamento-1866-edificio-los-pinos-iii-cercado-de-lima-lima-lima-grupo-mg"


df = scraping_selenium(url)

# Actualización de la función para manejar los dos patrones adicionales
def limpiar_titulo_actualizado(titulo):
        # Buscar número de unidades disponibles
        #unidades = re.search(r'(\d+) unidades?', titulo)
        unidades = re.search(r'(\d+) unidad?', titulo)
        unidades_disponibles = int(unidades.group(1)) if unidades else None
        
        # Buscar el modelo (secuencia de palabras hasta 'Piso')
        modelo = re.search(r'(\w+)\sPiso', titulo)
        modelo_nombre = modelo.group(1) if modelo else None

        # Caso 1: Patrón "Entre X al Y"
        # entre_pisos = re.search(r'Entre (\d+) al (\d+)', titulo)
        entre_pisos = re.search(r'Entre\s*(\d+)\s*al\s*(\d+)', titulo)
        if entre_pisos:
            piso_desde = int(entre_pisos.group(1))
            piso_hasta = int(entre_pisos.group(2))
            total_pisos = piso_hasta - piso_desde + 1
        else:
            # Caso 2: Patrón "Piso X, Y, Z"
            pisos_comas = re.findall(r'Piso (\d+(?:, \d+)*)', titulo)
            if pisos_comas:
                pisos = [int(piso) for piso in pisos_comas[0].split(', ')]
                piso_desde = min(pisos)
                piso_hasta = max(pisos)
                total_pisos = len(pisos)
            else:
                piso_desde, piso_hasta, total_pisos = None, None, None

        # Buscar dormitorios
        dormitorios = re.search(r'(\d+) Dormitorios?', titulo)
        dormitorios_num = int(dormitorios.group(1)) if dormitorios else None
        
        # Buscar área
        area = re.search(r'Área ([\d.]+)', titulo)
        area_m2 = float(area.group(1)) if area else None
        
        # Buscar precio
        precio = re.search(r'Precio desde S/ ([\d,]+)', titulo)
        precio_soles = float(precio.group(1).replace(',', '')) if precio else None
        
        return [unidades_disponibles, modelo_nombre, piso_desde, piso_hasta, total_pisos, dormitorios_num, area_m2, precio_soles]

    # Aplicar la función actualizada

df_comas_cleaned_actualizado = df['TITULO'].apply(limpiar_titulo_actualizado)
df_comas_cleaned_actualizado = pd.DataFrame(df_comas_cleaned_actualizado.tolist(), columns=[
    'Unidades_Disponibles', 'Modelo', 'Piso_Desde', 'Piso_Hasta', 'Total_Pisos', 'Dormitorios', 'Area_m2', 'Precio_Desde_Soles'
])

df_comas_cleaned_actualizado = df_comas_cleaned_actualizado.dropna(axis=0, how='all')

#Imagina
#exportar_excel(df_comas_cleaned_actualizado, nombre_file="Lima Proyecto", proyecto = "Alto Lima Park 16_04_25")
#exportar_excel(df_comas_cleaned_actualizado, nombre_file="Lima Proyecto", proyecto = "Alto Lima Park 05_05_25")

#Abril
#exportar_excel(df_comas_cleaned_actualizado, nombre_file="Lima Proyecto", proyecto = "Cedro 33 16_04_25")
#exportar_excel(df_comas_cleaned_actualizado, nombre_file="Lima Proyecto", proyecto = "Cedro 33 05_05_25")

#Alameda
#exportar_excel(df_comas_cleaned_actualizado, nombre_file="Lima Proyecto", proyecto = "Alameda Central 16_04_25")
#exportar_excel(df_comas_cleaned_actualizado, nombre_file="Lima Proyecto", proyecto = "Alameda Central 05_05_05")

# Los Portales
#exportar_excel(df_comas_cleaned_actualizado, nombre_file="Lima Proyecto", proyecto = "Gran Central Colonial II 16_04_25")
exportar_excel(df_comas_cleaned_actualizado, nombre_file="Lima Proyecto", proyecto = "Gran Central Colonial II 05_05_25")

#exportar_excel(df_comas_cleaned_actualizado, nombre_file="Lima Proyecto", proyecto = "Albamar 16_04_25")
#exportar_excel(df_comas_cleaned_actualizado, nombre_file="Lima Proyecto", proyecto = "Céntriqo 16_04_25")
#exportar_excel(df_comas_cleaned_actualizado, nombre_file="Lima Proyecto", proyecto = "Céntriqo 05_05_05")
#exportar_excel(df_comas_cleaned_actualizado, nombre_file="Lima Proyecto", proyecto = "Alto Alborada 16_04_25")

#exportar_excel(df_comas_cleaned_actualizado, nombre_file="Lima Proyecto", proyecto = "Hanan")
#exportar_excel(df_comas_cleaned_actualizado, nombre_file="Lima Proyecto", proyecto = "Duetto 846")
#exportar_excel(df_comas_cleaned_actualizado, nombre_file="Lima Proyecto", proyecto = "Urban Tower 3")
#exportar_excel(df_comas_cleaned_actualizado, nombre_file="Lima Proyecto", proyecto = "Evolution")
#exportar_excel(df_comas_cleaned_actualizado, nombre_file="Lima Proyecto", proyecto = "Amatista")
#exportar_excel(df_comas_cleaned_actualizado, nombre_file="Lima Proyecto", proyecto = "")


# Mostrar el DataFrame resultante
print(df.head())

### VERSIÓN OOP

In [8]:
import re
import pandas as pd
from bs4 import BeautifulSoup as BS
from selenium import webdriver

class MarketScraper:
    """
    Clase para extracción, limpieza y análisis de datos de proyectos inmobiliarios.
    """
    def __init__(self, url=None, df=None):
        self.url = url
        self.df = df
        self.soup = None

    def _init_driver(self):
        driver = webdriver.Chrome()
        driver.get(self.url)
        content = driver.page_source
        driver.quit()
        self.soup = BS(content, 'html.parser')

    def extract_data(self, tag: str, attrs: dict) -> list:
        """
        Extrae texto de etiquetas HTML según tag y atributos.
        """
        items = self.soup.find_all(tag, attrs=attrs)
        return [el.text.strip() for el in items]

    def normalize_columns(self):
        """
        Convierte nombres de columnas a mayúsculas.
        """
        self.df.columns = self.df.columns.str.upper()
        return self

    def _build_regex(self, before: str = None, after: str = None, price: bool = False) -> str:
        if price:
            num = r"\d{1,3}(?:,\d{3})*"
        else:
            num = r"\d+"
        if before and after:
            return rf"{re.escape(after)}\s*({num})\s*{re.escape(before)}"
        if after:
            return rf"{re.escape(after)}\s*({num})"
        if before:
            return rf"({num})\s*{re.escape(before)}"
        raise ValueError("Se requiere al menos 'before' o 'after'.")

    def find_number(self, col: str, before: str = None, after: str = None, price: bool = False) -> pd.Series:
        """
        Extrae números de una columna según patrones opcionales.
        """
        regex = self._build_regex(before, after, price)
        def _extract(x):
            m = re.search(regex, str(x))
            if m:
                val = m.group(1)
                return int(val.replace(',', '')) if price else int(val)
            return None
        return self.df[col].apply(_extract)

    def clean_price(self, col: str = 'PRECIO_DESDE'):
        """
        Extrae y convierte precios en soles y dólares.
        """
        s = self.df[col].astype(str)
        self.df['Moneda_soles'] = s.str.contains('S/').map({True: 'Soles'})
        self.df['Moneda_dolares'] = s.str.contains('USD').map({True: 'Dólares'})
        # extraer valores
        self.df['Precio_soles'] = s.where(self.df['Moneda_soles'].notna()) \
            .str.extract(r"(\d+)\b")[0].astype(float) * 1000
        self.df['Precio_dolares'] = self.find_number(col, after='USD', price=True)
        return self

    def clean_location(self, col: str = 'UBICACION', districts: list = None):
        """
        Elimina distritos y limpia texto de ubicación.
        """
        if districts is None:
            districts = ['Comas','Breña','Lima','Peru','Perú','San Miguel']
        def _clean(text):
            if not isinstance(text, str):
                return None
            for d in districts:
                text = re.sub(rf"\b{d}\b", '', text, flags=re.IGNORECASE)
            text = re.sub(r"[\.,;:!?]+$", '', text).strip()
            return re.sub(r"\s+", ' ', text) or None
        self.df['Calle'] = self.df[col].apply(_clean)
        return self

    def extract_features(self, col: str = 'METRAJE'):
        """
        Extrae unidades, dormitorios, metraje, baños y estacionamientos.
        """
        self.df['Unidades'] = self.find_number(col, before='un.')
        self.df['Dormitorios'] = self.find_number(col, before='dorm.')
        self.df['Metraje_desde'] = self.find_number(col, before='a', after='dorm.')
        self.df['Metraje_hasta'] = self.find_number(col, before='m²')
        self.df['Baños'] = self.find_number(col, before='baño')
        self.df['Estacionamientos'] = self.find_number(col, before='esta')
        for c in ['Unidades','Dormitorios','Metraje_desde','Metraje_hasta','Baños','Estacionamientos']:
            self.df[c] = pd.to_numeric(self.df[c], errors='coerce')
        return self

    def add_calculations(self):
        """
        Añade precios por m2 en soles y dólares.
        """
        self.df['Precio m2_soles_desde'] = (self.df['Precio_soles'] / self.df['Metraje_hasta']).round()
        self.df['Precio m2_soles_hasta'] = (self.df['Precio_soles'] / self.df['Metraje_desde']).round()
        if 'Precio_dolares' in self.df:
            self.df['Precio m2_usd_desde'] = (self.df['Precio_dolares'] / self.df['Metraje_hasta']).round()
            self.df['Precio m2_usd_hasta'] = (self.df['Precio_dolares'] / self.df['Metraje_desde']).round()
        return self

    def export_to_excel(self, filename: str, project: str):
        """
        Exporta DataFrame a Excel en ruta configurada.
        """
        base = "C:/Users/diego.dinatale/OneDrive - Aenza/LOCAL_MACHINE/DATA_ANALYTICS/"  
        path = f"{base}BD VIVA/DATA DEL MERCADO CON SCRAPPING/CERCADO DE LIMA/{filename} {project}.xlsx"
        print(f"Guardando en {path}")
        self.df.to_excel(path, index=False)
        print("Exportación exitosa.")
        return self

    def run(self, project: str, price_col: str = 'PRECIO_DESDE', loc_col: str = 'UBICACION', area_col: str = 'METRAJE') -> pd.DataFrame:
        """
        Flujo completo: scraping, limpieza, extracción de características y exportación.
        """
        self._init_driver()
        datos = self.extract_data('div', {'class': 'caption'})
        self.df = pd.DataFrame({'TITULO': datos})
        (self.normalize_columns()
             .clean_price(price_col)
             .clean_location(loc_col)
             .extract_features(area_col)
             .add_calculations()
             .export_to_excel('Datos_Scraping', project))
        return self.df


In [9]:

# Ejemplo de uso:
url1 = "https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2825-gran-central-colonial-ii-cercado-de-lima-lima-lima-los-portales-departamentos"

scraper = MarketScraper(url=url1)
df_final = scraper.run(project="Proyecto Gran Central Colonial II 26_05_25")
df_final

KeyError: 'PRECIO_DESDE'

### VERSION 2

In [12]:
import re
import pandas as pd
from bs4 import BeautifulSoup as BS
from selenium import webdriver

class MarketScraper:
    """
    Clase para extracción, limpieza y análisis de datos de proyectos inmobiliarios.
    """
    def __init__(self, url=None, df=None):
        self.url = url
        self.df = df
        self.soup = None

    def _init_driver(self):
        driver = webdriver.Chrome()
        driver.get(self.url)
        content = driver.page_source
        driver.quit()
        self.soup = BS(content, 'html.parser')

    def extract_data(self, tag: str, attrs: dict) -> list:
        """
        Extrae texto de etiquetas HTML según tag y atributos.
        """
        items = self.soup.find_all(tag, attrs=attrs)
        return [el.text.strip() for el in items]

    def normalize_columns(self):
        """
        Convierte nombres de columnas a mayúsculas.
        """
        self.df.columns = self.df.columns.str.upper()
        return self

    def extract_title_features(self, title_col: str = 'TITULO'):
        """
        Extrae información adicional del título: unidades, modelo, pisos, dormitorios, área y precio.
        """
        def _parse(titulo):
            # Unidades disponibles
            u = re.search(r'(\d+) unidad?', titulo)
            unidades = int(u.group(1)) if u else None
            # Modelo
            m = re.search(r'(\w+)\sPiso', titulo)
            modelo = m.group(1) if m else None
            # Rangos de pisos
            ep = re.search(r'Entre\s*(\d+)\s*al\s*(\d+)', titulo)
            if ep:
                piso_desde = int(ep.group(1)); piso_hasta = int(ep.group(2))
                total_pisos = piso_hasta - piso_desde + 1
            else:
                pc = re.findall(r'Piso (\d+(?:, \d+)*)', titulo)
                if pc:
                    pisos_list = [int(p) for p in pc[0].split(', ')]
                    piso_desde = min(pisos_list); piso_hasta = max(pisos_list)
                    total_pisos = len(pisos_list)
                else:
                    piso_desde = piso_hasta = total_pisos = None
            # Dormitorios
            d = re.search(r'(\d+) Dormitorios?', titulo)
            dormitorios = int(d.group(1)) if d else None
            # Área m2
            a = re.search(r'Área ([\d.]+)', titulo)
            area = float(a.group(1)) if a else None
            # Precio desde
            p = re.search(r'Precio desde S/ ([\d,]+)', titulo)
            precio = float(p.group(1).replace(',', '')) if p else None
            return [unidades, modelo, piso_desde, piso_hasta, total_pisos, dormitorios, area, precio]
        # Aplicar y unir al DataFrame
        cols = ['Unidades_Disponibles', 'Modelo', 'Piso_Desde', 'Piso_Hasta', 'Total_Pisos',
                'Dormitorios', 'Area_m2', 'Precio_Desde_Soles']
        title_df = pd.DataFrame(self.df[title_col].astype(str).apply(_parse).tolist(), columns=cols)
        self.df = pd.concat([self.df, title_df], axis=1)
        # Eliminar filas vacías completas
        self.df.dropna(how='all', subset=cols, inplace=True)
        return self

    def _build_regex(self, before: str = None, after: str = None, price: bool = False) -> str:
        if price:
            num = r"\d{1,3}(?:,\d{3})*"
        else:
            num = r"\d+"
        if before and after:
            return rf"{re.escape(after)}\s*({num})\s*{re.escape(before)}"
        if after:
            return rf"{re.escape(after)}\s*({num})"
        if before:
            return rf"({num})\s*{re.escape(before)}"
        raise ValueError("Se requiere al menos 'before' o 'after'.")

    def find_number(self, col: str, before: str = None, after: str = None, price: bool = False) -> pd.Series:
        """
        Extrae números de una columna según patrones opcionales.
        """
        regex = self._build_regex(before, after, price)
        def _extract(x):
            m = re.search(regex, str(x))
            if m:
                val = m.group(1)
                return int(val.replace(',', '')) if price else int(val)
            return None
        return self.df[col].apply(_extract)

    def clean_price(self, col: str = 'PRECIO_DESDE'):
        """
        Extrae y convierte precios en soles y dólares.
        """
        s = self.df[col].astype(str)
        self.df['Moneda_soles'] = s.str.contains('S/').map({True: 'Soles'})
        self.df['Moneda_dolares'] = s.str.contains('USD').map({True: 'Dólares'})
        self.df['Precio_soles'] = s.where(self.df['Moneda_soles'].notna()) \
            .str.extract(r"(\d+)\b")[0].astype(float) * 1000
        self.df['Precio_dolares'] = self.find_number(col, after='USD', price=True)
        return self

    def clean_location(self, col: str = 'UBICACION', districts: list = None):
        """
        Elimina distritos y limpia texto de ubicación.
        """
        if districts is None:
            districts = ['Comas','Breña','Lima','Peru','Perú','San Miguel']
        def _clean(text):
            if not isinstance(text, str):
                return None
            for d in districts:
                text = re.sub(rf"\b{d}\b", '', text, flags=re.IGNORECASE)
            text = re.sub(r"[\.,;:!?]+$", '', text).strip()
            return re.sub(r"\s+", ' ', text) or None
        self.df['Calle'] = self.df[col].apply(_clean)
        return self

    def extract_features(self, col: str = 'METRAJE'):
        """
        Extrae unidades, dormitorios, metraje, baños y estacionamientos.
        """
        self.df['Unidades'] = self.find_number(col, before='un.')
        self.df['Dormitorios'] = self.find_number(col, before='dorm.')
        self.df['Metraje_desde'] = self.find_number(col, before='a', after='dorm.')
        self.df['Metraje_hasta'] = self.find_number(col, before='m²')
        self.df['Baños'] = self.find_number(col, before='baño')
        self.df['Estacionamientos'] = self.find_number(col, before='esta')
        for c in ['Unidades','Dormitorios','Metraje_desde','Metraje_hasta','Baños','Estacionamientos']:
            self.df[c] = pd.to_numeric(self.df[c], errors='coerce')
        return self

    def add_calculations(self):
        """
        Añade precios por m2 en soles y dólares.
        """
        self.df['Precio m2_soles_desde'] = (self.df['Precio_soles'] / self.df['Metraje_hasta']).round()
        self.df['Precio m2_soles_hasta'] = (self.df['Precio_soles'] / self.df['Metraje_desde']).round()
        if 'Precio_dolares' in self.df:
            self.df['Precio m2_usd_desde'] = (self.df['Precio_dolares'] / self.df['Metraje_hasta']).round()
            self.df['Precio m2_usd_hasta'] = (self.df['Precio_dolares'] / self.df['Metraje_desde']).round()
        return self

    def export_to_excel(self, filename: str, project: str):
        """
        Exporta DataFrame a Excel en ruta configurada.
        """
        base = "C:/Users/diego.dinatale/OneDrive - Aenza/LOCAL_MACHINE/DATA_ANALYTICS/"
        path = f"{base}BD VIVA/DATA DEL MERCADO CON SCRAPPING/CERCADO DE LIMA/{filename} {project}.xlsx"
        print(f"Guardando en {path}")
        self.df.to_excel(path, index=False)
        print("Exportación exitosa.")
        return self

    def run(self, project: str, price_col: str = 'PRECIO_DESDE', loc_col: str = 'UBICACION', area_col: str = 'METRAJE') -> pd.DataFrame:
        """
        Flujo completo: scraping, limpieza, extracción de características y exportación.
        """
        self._init_driver()
        datos = self.extract_data('div', {'class': 'caption'})
        self.df = pd.DataFrame({'TITULO': datos})
        (self.normalize_columns()
             .extract_title_features()
             .clean_price(price_col)
             .clean_location(loc_col)
             .extract_features(area_col)
             .add_calculations()
             .export_to_excel('Datos_Scraping', project))
        return self.df


In [13]:
# Ejemplo de uso:
# scraper = MarketScraper(url="https://ejemplo.com")
# df_final = scraper.run(project="ProyectoX")

# Ejemplo de uso:
url1 = "https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2825-gran-central-colonial-ii-cercado-de-lima-lima-lima-los-portales-departamentos"

scraper = MarketScraper(url=url1)
df_final = scraper.run(project="Proyecto1")
df_final

KeyError: 'PRECIO_DESDE'

### VERSION 3

In [14]:
import re
import pandas as pd
from bs4 import BeautifulSoup as BS
from selenium import webdriver

class MarketScraper:
    """
    Clase para extracción, limpieza y análisis de datos de proyectos inmobiliarios.
    """
    def __init__(self, url=None, df=None):
        self.url = url
        self.df = df
        self.soup = None

    def _init_driver(self):
        driver = webdriver.Chrome()
        driver.get(self.url)
        content = driver.page_source
        driver.quit()
        self.soup = BS(content, 'html.parser')

    def extract_data(self, tag: str, attrs: dict) -> list:
        """
        Extrae texto de etiquetas HTML según tag y atributos.
        """
        items = self.soup.find_all(tag, attrs=attrs)
        return [el.text.strip() for el in items]

    def normalize_columns(self):
        """
        Convierte nombres de columnas a mayúsculas.
        """
        self.df.columns = self.df.columns.str.upper()
        return self

    def extract_title_features(self, title_col: str = 'TITULO'):
        """
        Extrae información adicional del título: unidades, modelo, pisos, dormitorios, área y precio.
        """
        def _parse(titulo):
            u = re.search(r'(\d+) unidad?', titulo)
            unidades = int(u.group(1)) if u else None
            m = re.search(r'(\w+)\sPiso', titulo)
            modelo = m.group(1) if m else None
            ep = re.search(r'Entre\s*(\d+)\s*al\s*(\d+)', titulo)
            if ep:
                piso_desde, piso_hasta = int(ep.group(1)), int(ep.group(2))
                total_pisos = piso_hasta - piso_desde + 1
            else:
                pc = re.findall(r'Piso (\d+(?:, \d+)*)', titulo)
                if pc:
                    pisos_list = [int(p) for p in pc[0].split(', ')]
                    piso_desde, piso_hasta = min(pisos_list), max(pisos_list)
                    total_pisos = len(pisos_list)
                else:
                    piso_desde = piso_hasta = total_pisos = None
            d = re.search(r'(\d+) Dormitorios?', titulo)
            dormitorios = int(d.group(1)) if d else None
            a = re.search(r'Área ([\d.]+)', titulo)
            area = float(a.group(1)) if a else None
            p = re.search(r'Precio desde S/ ([\d,]+)', titulo)
            precio = float(p.group(1).replace(',', '')) if p else None
            return [unidades, modelo, piso_desde, piso_hasta, total_pisos, dormitorios, area, precio]
        cols = ['Unidades_Disponibles', 'Modelo', 'Piso_Desde', 'Piso_Hasta', 'Total_Pisos',
                'Dormitorios', 'Area_m2', 'Precio_Desde_Soles']
        title_df = pd.DataFrame(self.df[title_col].astype(str).apply(_parse).tolist(), columns=cols)
        self.df = pd.concat([self.df, title_df], axis=1)
        self.df.dropna(how='all', subset=cols, inplace=True)
        return self

    def _build_regex(self, before: str = None, after: str = None, price: bool = False) -> str:
        if price:
            num = r"\d{1,3}(?:,\d{3})*"
        else:
            num = r"\d+"
        if before and after:
            return rf"{re.escape(after)}\s*({num})\s*{re.escape(before)}"
        if after:
            return rf"{re.escape(after)}\s*({num})"
        if before:
            return rf"({num})\s*{re.escape(before)}"
        raise ValueError("Se requiere al menos 'before' o 'after'.")

    def find_number(self, col: str, before: str = None, after: str = None, price: bool = False) -> pd.Series:
        regex = self._build_regex(before, after, price)
        def _extract(x):
            m = re.search(regex, str(x))
            if m:
                val = m.group(1)
                return int(val.replace(',', '')) if price else int(val)
            return None
        return self.df[col].apply(_extract)

    def clean_price(self, col: str = 'PRECIO_DESDE'):
        """
        Extrae y convierte precios en soles y dólares si la columna existe.
        """
        if col not in self.df.columns:
            return self
        s = self.df[col].astype(str)
        self.df['Moneda_soles'] = s.str.contains('S/').map({True: 'Soles', False: None})
        self.df['Moneda_dolares'] = s.str.contains('USD').map({True: 'Dólares', False: None})
        self.df['Precio_soles'] = s.where(self.df['Moneda_soles'].notnull()) \
            .str.extract(r"(\d+)\b")[0].astype(float) * 1000
        self.df['Precio_dolares'] = self.find_number(col, after='USD', price=True)
        return self

    def clean_location(self, col: str = 'UBICACION', districts: list = None):
        if districts is None:
            districts = ['Comas','Breña','Lima','Peru','Perú','San Miguel']
        def _clean(text):
            if not isinstance(text, str):
                return None
            for d in districts:
                text = re.sub(rf"\b{d}\b", '', text, flags=re.IGNORECASE)
            return re.sub(r"\s+", ' ', re.sub(r"[\.,;:!?]+$", '', text)).strip() or None
        self.df['Calle'] = self.df[col].apply(_clean)
        return self

    def extract_features(self, col: str = 'METRAJE'):
        keys = ['Unidades','Dormitorios','Metraje_desde','Metraje_hasta','Baños','Estacionamientos']
        patterns = [('un.', None), ('dorm.', None), ('a', 'dorm.'), ('m²', None), ('baño', None), ('esta', None)]
        for key, (before, after) in zip(keys, patterns):
            self.df[key] = pd.to_numeric(self.find_number(col, before=before, after=after), errors='coerce')
        return self

    def add_calculations(self):
        if 'Precio_soles' in self.df and 'Metraje_hasta' in self.df:
            self.df['Precio m2_soles_desde'] = (self.df['Precio_soles'] / self.df['Metraje_hasta']).round()
        if 'Precio_soles' in self.df and 'Metraje_desde' in self.df:
            self.df['Precio m2_soles_hasta'] = (self.df['Precio_soles'] / self.df['Metraje_desde']).round()
        if 'Precio_dolares' in self.df and 'Metraje_hasta' in self.df:
            self.df['Precio m2_usd_desde'] = (self.df['Precio_dolares'] / self.df['Metraje_hasta']).round()
        if 'Precio_dolares' in self.df and 'Metraje_desde' in self.df:
            self.df['Precio m2_usd_hasta'] = (self.df['Precio_dolares'] / self.df['Metraje_desde']).round()
        return self

    def export_to_excel(self, filename: str, project: str):
        base = "C:/Users/diego.dinatale/OneDrive - Aenza/LOCAL_MACHINE/DATA_ANALYTICS/"
        path = f"{base}BD VIVA/DATA DEL MERCADO CON SCRAPPING/CERCADO DE LIMA/{filename} {project}.xlsx"
        print(f"Guardando en {path}")
        self.df.to_excel(path, index=False)
        print("Exportación exitosa.")
        return self

    def run(self, project: str, price_col: str = 'PRECIO_DESDE', loc_col: str = 'UBICACION', area_col: str = 'METRAJE') -> pd.DataFrame:
        self._init_driver()
        datos = self.extract_data('div', {'class': 'caption'})
        self.df = pd.DataFrame({'TITULO': datos})
        (self.normalize_columns()
             .extract_title_features()
             .clean_price(price_col)
             .clean_location(loc_col)
             .extract_features(area_col)
             .add_calculations()
             .export_to_excel('Datos_Scraping', project))
        return self.df

# Ejemplo de uso:
# scraper = MarketScraper(url="https://ejemplo.com")
# df_final = scraper.run(project="ProyectoX")

In [15]:

# Ejemplo de uso:
url1 = "https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2825-gran-central-colonial-ii-cercado-de-lima-lima-lima-los-portales-departamentos"

scraper = MarketScraper(url=url1)
df_final = scraper.run(project="Proyecto1")
df_final

KeyError: 'UBICACION'

### VERSION 4

In [16]:
import re
import pandas as pd
from bs4 import BeautifulSoup as BS
from selenium import webdriver

class MarketScraper:
    """
    Clase para extracción, limpieza y análisis de datos de proyectos inmobiliarios.
    """
    def __init__(self, url=None, df=None):
        self.url = url
        self.df = df
        self.soup = None

    def _init_driver(self):
        driver = webdriver.Chrome()
        driver.get(self.url)
        content = driver.page_source
        driver.quit()
        self.soup = BS(content, 'html.parser')

    def extract_data(self, tag: str, attrs: dict) -> list:
        items = self.soup.find_all(tag, attrs=attrs)
        return [el.text.strip() for el in items]

    def normalize_columns(self):
        self.df.columns = self.df.columns.str.upper()
        return self

    def extract_title_features(self, title_col: str = 'TITULO'):
        def _parse(titulo):
            u = re.search(r'(\d+) unidad?', titulo)
            unidades = int(u.group(1)) if u else None
            m = re.search(r'(\w+)\sPiso', titulo)
            modelo = m.group(1) if m else None
            ep = re.search(r'Entre\s*(\d+)\s*al\s*(\d+)', titulo)
            if ep:
                piso_desde, piso_hasta = int(ep.group(1)), int(ep.group(2))
                total_pisos = piso_hasta - piso_desde + 1
            else:
                pc = re.findall(r'Piso (\d+(?:, \d+)*)', titulo)
                if pc:
                    pisos_list = [int(p) for p in pc[0].split(', ')]
                    piso_desde, piso_hasta = min(pisos_list), max(pisos_list)
                    total_pisos = len(pisos_list)
                else:
                    piso_desde = piso_hasta = total_pisos = None
            d = re.search(r'(\d+) Dormitorios?', titulo)
            dormitorios = int(d.group(1)) if d else None
            a = re.search(r'Área ([\d.]+)', titulo)
            area = float(a.group(1)) if a else None
            p = re.search(r'Precio desde S/ ([\d,]+)', titulo)
            precio = float(p.group(1).replace(',', '')) if p else None
            return [unidades, modelo, piso_desde, piso_hasta, total_pisos, dormitorios, area, precio]
        cols = ['Unidades_Disponibles', 'Modelo', 'Piso_Desde', 'Piso_Hasta', 'Total_Pisos',
                'Dormitorios', 'Area_m2', 'Precio_Desde_Soles']
        title_df = pd.DataFrame(self.df[title_col].astype(str).apply(_parse).tolist(), columns=cols)
        self.df = pd.concat([self.df, title_df], axis=1)
        self.df.dropna(how='all', subset=cols, inplace=True)
        return self

    def _build_regex(self, before: str = None, after: str = None, price: bool = False) -> str:
        num = r"\d{1,3}(?:,\d{3})*" if price else r"\d+"
        if before and after:
            return rf"{re.escape(after)}\s*({num})\s*{re.escape(before)}"
        if after:
            return rf"{re.escape(after)}\s*({num})"
        if before:
            return rf"({num})\s*{re.escape(before)}"
        raise ValueError("Se requiere al menos 'before' o 'after'.")

    def find_number(self, col: str, before: str = None, after: str = None, price: bool = False) -> pd.Series:
        regex = self._build_regex(before, after, price)
        return self.df[col].apply(lambda x: int(re.search(regex, str(x)).group(1).replace(',', ''))
                                  if re.search(regex, str(x)) else None)

    def clean_price(self, col: str = 'PRECIO_DESDE'):
        if col in self.df.columns:
            s = self.df[col].astype(str)
            self.df['Moneda_soles'] = s.str.contains('S/').map({True: 'Soles', False: None})
            self.df['Moneda_dolares'] = s.str.contains('USD').map({True: 'Dólares', False: None})
            self.df['Precio_soles'] = s.where(self.df['Moneda_soles'].notnull()) \
                .str.extract(r"(\d+)\b")[0].astype(float) * 1000
            self.df['Precio_dolares'] = self.find_number(col, after='USD', price=True)
        return self

    def clean_location(self, col: str = 'UBICACION', districts: list = None):
        if col in self.df.columns:
            if districts is None:
                districts = ['Comas', 'Breña', 'Lima', 'Peru', 'Perú', 'San Miguel']
            def _clean(text):
                if not isinstance(text, str):
                    return None
                for d in districts:
                    text = re.sub(rf"\b{d}\b", '', text, flags=re.IGNORECASE)
                return re.sub(r"\s+", ' ', re.sub(r"[\.,;:!?]+$", '', text)).strip() or None
            self.df['Calle'] = self.df[col].apply(_clean)
        return self

    def extract_features(self, col: str = 'METRAJE'):
        if col in self.df.columns:
            keys = ['Unidades', 'Dormitorios', 'Metraje_desde', 'Metraje_hasta', 'Baños', 'Estacionamientos']
            patterns = [('un.', None), ('dorm.', None), ('a', 'dorm.'), ('m²', None), ('baño', None), ('esta', None)]
            for key, (before, after) in zip(keys, patterns):
                self.df[key] = pd.to_numeric(self.find_number(col, before=before, after=after), errors='coerce')
        return self

    def add_calculations(self):
        if 'Precio_soles' in self.df and 'Metraje_hasta' in self.df:
            self.df['Precio m2_soles_desde'] = (self.df['Precio_soles'] / self.df['Metraje_hasta']).round()
        if 'Precio_soles' in self.df and 'Metraje_desde' in self.df:
            self.df['Precio m2_soles_hasta'] = (self.df['Precio_soles'] / self.df['Metraje_desde']).round()
        if 'Precio_dolares' in self.df and 'Metraje_hasta' in self.df:
            self.df['Precio m2_usd_desde'] = (self.df['Precio_dolares'] / self.df['Metraje_hasta']).round()
        if 'Precio_dolares' in self.df and 'Metraje_desde' in self.df:
            self.df['Precio m2_usd_hasta'] = (self.df['Precio_dolares'] / self.df['Metraje_desde']).round()
        return self

    def export_to_excel(self, filename: str, project: str):
        base = "C:/Users/diego.dinatale/OneDrive - Aenza/LOCAL_MACHINE/DATA_ANALYTICS/"
        path = f"{base}BD VIVA/DATA DEL MERCADO CON SCRAPPING/CERCADO DE LIMA/{filename} {project}.xlsx"
        print(f"Guardando en {path}")
        self.df.to_excel(path, index=False)
        print("Exportación exitosa.")
        return self

    def run(self, project: str, price_col: str = 'PRECIO_DESDE', loc_col: str = 'UBICACION', area_col: str = 'METRAJE') -> pd.DataFrame:
        self._init_driver()
        datos = self.extract_data('div', {'class': 'caption'})
        self.df = pd.DataFrame({'TITULO': datos})
        (self.normalize_columns()
             .extract_title_features()
             .clean_price(price_col)
             .clean_location(loc_col)
             .extract_features(area_col)
             .add_calculations()
             .export_to_excel('Datos_Scraping', project))
        return self.df

# Ejemplo de uso:
# scraper = MarketScraper(url="https://ejemplo.com")
# df_final = scraper.run(project="ProyectoX")


In [17]:

# Ejemplo de uso:
url1 = "https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2825-gran-central-colonial-ii-cercado-de-lima-lima-lima-los-portales-departamentos"

scraper = MarketScraper(url=url1)
df_final = scraper.run(project="Proyecto1")
df_final

Guardando en C:/Users/diego.dinatale/OneDrive - Aenza/LOCAL_MACHINE/DATA_ANALYTICS/BD VIVA/DATA DEL MERCADO CON SCRAPPING/CERCADO DE LIMA/Datos_Scraping Proyecto1.xlsx
Exportación exitosa.


,TITULO,Unidades_Disponibles,Modelo,Piso_Desde,Piso_Hasta,Total_Pisos,Dormitorios,Area_m2,Precio_Desde_Soles


In [ ]:
import re
import pandas as pd
import datetime
from bs4 import BeautifulSoup as BS
from selenium import webdriver

class MarketScraper:
    """
    Clase para extracción, limpieza y análisis de datos de proyectos inmobiliarios.
    """
    def __init__(self, url=None, df=None):
        self.url = url
        self.df = df
        self.soup = None

    def _init_driver(self):
        driver = webdriver.Chrome()
        driver.get(self.url)
        content = driver.page_source
        driver.quit()
        self.soup = BS(content, 'html.parser')

    def extract_data(self, tag: str, attrs: dict) -> list:
        items = self.soup.find_all(tag, attrs=attrs)
        return [el.text.strip() for el in items]

    def normalize_columns(self):
        self.df.columns = self.df.columns.str.upper()
        return self

    def debug_dormitorios(self) -> list:
        matches = self.soup.find_all(text=re.compile(r'Dormitoris?', re.IGNORECASE))
        return [str(m.parent) for m in matches]

    def extract_title_features(self, title_col: str = 'TITULO'):
        def _parse(titulo):
            u = re.search(r'(\d+) unidad?', titulo)
            unidades = int(u.group(1)) if u else None
            m = re.search(r'(\w+)\sPiso', titulo)
            modelo = m.group(1) if m else None
            ep = re.search(r'Entre\s*(\d+)\s*al\s*(\d+)', titulo)
            if ep:
                piso_desde, piso_hasta = int(ep.group(1)), int(ep.group(2))
                total_pisos = piso_hasta - piso_desde + 1
            else:
                pc = re.findall(r'Piso (\d+(?:, \d+)*)', titulo)
                if pc:
                    pisos_list = [int(p) for p in pc[0].split(', ')]
                    piso_desde, piso_hasta = min(pisos_list), max(pisos_list)
                    total_pisos = len(pisos_list)
                else:
                    piso_desde = piso_hasta = total_pisos = None
            d = re.search(r'(\d+) Dormitorios?', titulo)
            dormitorios = int(d.group(1)) if d else None
            a = re.search(r'Área ([\d.]+)', titulo)
            area = float(a.group(1)) if a else None
            p = re.search(r'Precio desde S/ ([\d,]+)', titulo)
            precio = float(p.group(1).replace(',', '')) if p else None
            return [unidades, modelo, piso_desde, piso_hasta, total_pisos, dormitorios, area, precio]
        cols = ['Unidades_Disponibles', 'Modelo', 'Piso_Desde', 'Piso_Hasta', 'Total_Pisos',
                'Dormitorios', 'Area_m2', 'Precio_Desde_Soles']
        title_df = pd.DataFrame(self.df[title_col].astype(str).apply(_parse).tolist(), columns=cols)
        self.df = pd.concat([self.df, title_df], axis=1)
        self.df.dropna(how='all', subset=cols, inplace=True)
        return self

    def _build_regex(self, before: str = None, after: str = None, price: bool = False) -> str:
        num = r"\d{1,3}(?:,\d{3})*" if price else r"\d+"
        if before and after:
            return rf"{re.escape(after)}\s*({num})\s*{re.escape(before)}"
        if after:
            return rf"{re.escape(after)}\s*({num})"
        if before:
            return rf"({num})\s*{re.escape(before)}"
        raise ValueError("Se requiere al menos 'before' o 'after'.")

    def find_number(self, col: str, before: str = None, after: str = None, price: bool = False) -> pd.Series:
        regex = self._build_regex(before, after, price)
        return self.df[col].apply(lambda x: int(re.search(regex, str(x)).group(1).replace(',', ''))
                                  if re.search(regex, str(x)) else None)

    def clean_price(self, col: str = 'PRECIO_DESDE'):
        if col in self.df.columns:
            s = self.df[col].astype(str)
            self.df['Moneda_soles'] = s.str.contains('S/').map({True: 'Soles', False: None})
            self.df['Moneda_dolares'] = s.str.contains('USD').map({True: 'Dólares', False: None})
            self.df['Precio_soles'] = s.where(self.df['Moneda_soles'].notnull()) \
                .str.extract(r"(\d+)\b")[0].astype(float) * 1000
            self.df['Precio_dolares'] = self.find_number(col, after='USD', price=True)
        return self

    def clean_location(self, col: str = 'UBICACION', districts: list = None):
        if col in self.df.columns:
            if districts is None:
                districts = ['Comas', 'Breña', 'Lima', 'Peru', 'Perú', 'San Miguel']
            def _clean(text):
                if not isinstance(text, str):
                    return None
                for d in districts:
                    text = re.sub(rf"\b{d}\b", '', text, flags=re.IGNORECASE)
                return re.sub(r"\s+", ' ', re.sub(r"[\.,;:!?]+$", '', text)).strip() or None
            self.df['Calle'] = self.df[col].apply(_clean)
        return self

    def extract_features(self, col: str = 'METRAJE'):
        if col in self.df.columns:
            keys = ['Unidades', 'Dormitorios', 'Metraje_desde', 'Metraje_hasta', 'Baños', 'Estacionamientos']
            patterns = [('un.', None), ('dorm.', None), ('a', 'dorm.'), ('m²', None), ('baño', None), ('esta', None)]
            for key, (before, after) in zip(keys, patterns):
                self.df[key] = pd.to_numeric(self.find_number(col, before=before, after=after), errors='coerce')
        return self

    def add_calculations(self):
        if 'Precio_soles' in self.df and 'Metraje_hasta' in self.df:
            self.df['Precio m2_soles_desde'] = (self.df['Precio_soles'] / self.df['Metraje_hasta']).round()
        if 'Precio_soles' in self.df and 'Metraje_desde' in self.df:
            self.df['Precio m2_soles_hasta'] = (self.df['Precio_soles'] / self.df['Metraje_desde']).round()
        if 'Precio_dolares' in self.df and 'Metraje_hasta' in self.df:
            self.df['Precio m2_usd_desde'] = (self.df['Precio_dolares'] / self.df['Metraje_hasta']).round()
        if 'Precio_dolares' in self.df and 'Metraje_desde' in self.df:
            self.df['Precio m2_usd_hasta'] = (self.df['Precio_dolares'] / self.df['Metraje_desde']).round()
        return self

    def export_to_excel(self, filename: str, project: str):
        """
        Exporta DataFrame a Excel incluyendo fecha actual en nombre de archivo.
        """
        base = "C:/Users/diego.dinatale/OneDrive - Aenza/LOCAL_MACHINE/DATA_ANALYTICS/"
        fecha = datetime.datetime.now().strftime("%d_%m_%Y")
        nombre_file = f"{filename}_{fecha} {project}"
        path = f"{base}BD VIVA/DATA DEL MERCADO CON SCRAPPING/CERCADO especial/{nombre_file}.xlsx"
        print(f"Guardando en {path}")
        self.df.to_excel(path, index=False)
        print("Exportación exitosa.")
        return self

    def run(self, project: str, price_col: str = 'PRECIO_DESDE', loc_col: str = 'UBICACION', area_col: str = 'METRAJE') -> pd.DataFrame:
        self._init_driver()
        datos = self.extract_data('div', {'class': 'caption'})
        self.df = pd.DataFrame({'TITULO': datos})
        (self.normalize_columns()
             .extract_title_features()
             .clean_price(price_col)
             .clean_location(loc_col)
             .extract_features(area_col)
             .add_calculations()
             .export_to_excel('Datos_Scraping', project))
        return self.df
